In [ ]:
%matplotlib inline

import time
import shutil

import h5py
import numpy as np
import matplotlib.pyplot as plt

import torch

# f = h5py.File('../hists/bar_hughtok1_20180921_175004.h5', 'r')
shutil.copy('../hists/bar_hughtok1_20180921_175004.h5', '/tmp/foo.h5')
f = h5py.File('/tmp/foo.h5', 'r')
outputs = torch.from_numpy(f['outputs'][:])
labels = torch.from_numpy(f['labels'][:])
print(outputs.shape)

def plot_hist(x, bins, label=None):
    """
    from https://stackoverflow.com/questions/5328556/histogram-matplotlib/18611135#18611135
    """
    bins, edges = np.histogram(x, bins, normed=1)
    left,right = edges[:-1],edges[1:]
    X = np.array([left,right]).T.flatten()
    Y = np.array([bins,bins]).T.flatten()

    plt.plot(X,Y, label=label)

# for i in range(10):
#     c_idxes = (labels == i).view(-1).nonzero().view(-1).long()
#     _outputs = outputs[c_idxes]
#     for j in range(10):
#         plot_hist(_outputs[:, j].numpy(), bins=100, label=f'{j}')
#     plt.legend()
#     plt.show()


In [ ]:
N = outputs.size(0)
n = 128
outputs_ = outputs[-128:]
labels_ = labels[-128:]

for c in range(10):
    c_idxes = (labels_ == c).view(-1).nonzero().view(-1).long()
    c_n = c_idxes.size(0)
    c_labels = torch.zeros(c_n * 10, dtype=torch.int64)
    c_values = torch.zeros(c_n * 10, dtype=torch.float32)
    for j in range(10):
        b1 = j * c_n
        b2 = b1 + c_n
        c_labels[b1:b2] = j
        c_values[b1:b2] = outputs_[c_idxes, j]
    plt.scatter(c_labels.float().numpy(), c_values.numpy(), marker='_', color=np.random.rand(3), s=400, label=f'{c}')
    plt.show()

In [ ]:
import sklearn.metrics.cluster

"""
if we want to use mutual information, what distributions of labels do we want to compare?

I guess the predicted labels and the ground truth labels?
"""

_, pred = outputs_.max(dim=-1)
MI = sklearn.metrics.cluster.mutual_info_score(
    labels_true=labels_.numpy(),
    labels_pred=pred.numpy()
)
print('MI', MI)

adj_MI = sklearn.metrics.cluster.adjusted_mutual_info_score(
    labels_true=labels_.numpy(),
    labels_pred=pred.numpy()
)
print('adj_MI', adj_MI)
